In [1]:
import numpy as np
import skeliner as sk

In [2]:
seg_id = 720575940545220339
MESH_PATH = f"../data/{seg_id}.obj"
mesh = sk.io.load_mesh(MESH_PATH)

In [3]:
skel_auto = sk.skeletonize(
    mesh,
    detect_soma=True,        
    collapse_soma=True,
    bridge_gaps=True,
    prune_tiny_neurites=True,
    unit="nm",
    id=seg_id,
    verbose=True,
)

[skeliner] starting skeletonisation (636,684 vertices, 1,274,227 faces)
 ↳  build surface graph                          … 3.02 s
 ↳  bin surface vertices by geodesic distance    … 2.59 s
 ↳  compute bin centroids and radii              … 4.76 s
 ↳  post-skeletonization soma detection          … 0.16 s
      └─ Found soma at [951560.0, 1086484.6, 34101.5]
      └─ (r = 10148.5, 5344.6, 2575.0)
 ↳  map mesh faces to skeleton edges             … 0.50 s
 ↳  merge redundant near-soma nodes              … 0.40 s
      └─ 129 nodes merged into soma
      └─ Moved soma to [951519.2, 1086418.2, 34093.4]
      └─ (r =  9589.7, 6004.5, 2608.4)
 ↳  bridge skeleton gaps                         … 0.16 s
 ↳  build global minimum-spanning tree           … 0.09 s
 ↳  prune tiny neurites                          … 0.54 s
      └─ Merged 8 peri-soma nodes into soma 
      └─ Moved soma to [951589.5, 1086350.8, 34190.9]
      └─ (r =  8795.8, 5831.4, 2727.5)
TOTAL (soma + core + post)                    

In [ ]:


# Cache mesh vertices once – every manual stage needs them
mesh_vertices = np.asarray(mesh.vertices, dtype=np.float64)

# Fast pass: skip all optional stages (postprocess=False overrides detect_soma/collapse/etc.)
skel = sk.skeletonize(
    mesh,
    detect_soma=True,              # kept for readability; ignored when postprocess=False
    collapse_soma=True,
    bridge_gaps=True,
    prune_tiny_neurites=True,
    unit="nm",
    id=seg_id,
    verbose=True,
    postprocess=False,
)

# Stage 3: detect soma (defaults match the automatic pipeline)
skel = skel.detect_soma(
    radius_key="median",
    soma_radius_percentile_threshold=99.9,
    soma_radius_distance_factor=4.0,
    soma_min_nodes=3,
    verbose=True,
)

# Stage 5: collapse near-soma / fat nodes
skel = skel.merge_near_soma_nodes(
    mesh_vertices=mesh_vertices,
    radius_key="median",
    inside_tol=0.0,
    near_factor=1.2,
    fat_factor=0.20,
)

# Stage 6: bridge disconnected components (in-place)
skel.bridge_gaps(
    bridge_max_factor=None,        # adaptive heuristic
    bridge_recalc_after=None,
    rebuild_mst=False,             # defer MST rebuild to the next call
)

# Stage 7: global MST (identical to the automatic pass)
skel = skel.rebuild_mst()

# Stage 8: prune tiny neurites hugging the soma
skel = skel.prune_neurites(
    mesh_vertices=mesh_vertices,
    tip_extent_factor=1.2,
    stem_extent_factor=3.0,
    drop_single_node_branches=True,
)

[skeliner] starting skeletonisation (636,684 vertices, 1,274,227 faces)
 ↳  build surface graph                          … 2.19 s
 ↳  bin surface vertices by geodesic distance    … 2.68 s
 ↳  compute bin centroids and radii              … 4.95 s
 ↳  post-skeletonization soma detection          … 0.00 s
 ↳  map mesh faces to skeleton edges             … 0.52 s
TOTAL (soma + core)                              … 10.35 s (0.00 + 10.35)
[skeliner] detect_soma – merged 12 nodes → soma @ [951533.2, 1087241.3, 34006.7], r ≈ 3601.2


In [5]:
skel.soma

Soma(center=array([ 951622.06150595, 1086542.58208208,   34111.06247915]), axes=array([8526.84460745, 5767.81061719, 2742.94832458]), R=array([[-0.42056526,  0.83380586, -0.35762082],
       [ 0.82805199,  0.51384712,  0.22425664],
       [-0.37074893,  0.20181408,  0.90654085]]), verts=array([224868, 224999, 225000, ..., 419442, 419443, 419882],
      shape=(107892,)))

In [9]:
len(skel_auto.nodes), len(skel.nodes)

(11558, 11525)

In [7]:
skel.connectivity()

True

In [8]:
skel_auto.connectivity()

True